In [16]:
import pandas as pd
import numpy as np
import pickle as pkl

In [83]:
# shuffle = True

results = pkl.load(open("../results/crash_2/model_selection_shuffle_True.pkl", "rb"))

In [84]:
for learner, learner_results in results.items():
    print(learner)
    
    for sec in ["if_pehe", "pseudo_outcome_r", "pseudo_outcome_dr"]:
        
        print(learner, sec, np.mean(learner_results[sec]), np.std(learner_results[sec]))
    

XLearner
XLearner if_pehe -3.9102465458572624 22.5894565727775
XLearner pseudo_outcome_r 0.3241320898120587 0.004810642608310448
XLearner pseudo_outcome_dr 0.6959794999462814 0.013465551404811836
DRLearner
DRLearner if_pehe -4.142917499456462 52.754798623940566
DRLearner pseudo_outcome_r 0.3243910679816027 0.0041920187797287135
DRLearner pseudo_outcome_dr 0.6969906163554714 0.01201185506632268
SLearner
SLearner if_pehe -34.326718605677705 46.15804938755527
SLearner pseudo_outcome_r 0.3224761735800485 0.004249139776918623
SLearner pseudo_outcome_dr 0.6928256318922735 0.012083831705407885
TLearner
TLearner if_pehe -15.517754049884228 32.07485494970943
TLearner pseudo_outcome_r 0.32358320075669 0.005062191217803197
TLearner pseudo_outcome_dr 0.6954798365433397 0.013383696449574392
RLearner
RLearner if_pehe -33.043616043915605 38.356661673799145
RLearner pseudo_outcome_r 0.3221297874753258 0.004320118161976634
RLearner pseudo_outcome_dr 0.6921562622162268 0.01250315594382006
RALearner
RALe